In [15]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

#import time
#import argparse
#import multiprocessing


from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
#from IPython.display import display

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')

In [16]:
# This is needed to display the images.
%matplotlib inline

In [17]:
from object_detection.utils import label_map_util
from object_detection.utils import ops as utils_ops
from object_detection.utils import visualization_utils as vis_util

In [18]:
MODEL_NAME = 'inference_graph/ssdlite_mobilenet_v2_coco_33104'
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = os.path.join('annotations', 'labelmap-all.pbtxt')

In [19]:
NUM_CLASSES = 5

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES)
category_index = label_map_util.create_category_index(categories)

In [20]:
def run_inference_for_single_image(image, graph):
    if 'detection_masks' in tensor_dict:
        # Processamento de uma única imagem
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # É necessário atualizar a imagem para converter a máscara das coordenadas da caixa em coordenadas da imagem e ajustar o tamanho da imagem.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
    image_tensor = tf.compat.v1.get_default_graph().get_tensor_by_name('image_tensor:0')
    #print(image_tensor.eval, "\n")

    # Run inference
    output_dict = sess.run(tensor_dict,
                            feed_dict={image_tensor: np.expand_dims(image, 0)})
    #print(output_dict, "\n")
    # all outputs are float32 numpy arrays, so convert types as appropriate
    output_dict['num_detections'] = int(output_dict['num_detections'][0])
    output_dict['detection_classes'] = output_dict[
        'detection_classes'][0].astype(np.uint8)
    output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
    output_dict['detection_scores'] = output_dict['detection_scores'][0]
    if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

In [21]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.compat.v1.GraphDef()
  with tf.compat.v2.io.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [22]:
#import glob

import pandas as pd

#targetPattern = r"CSGO_images\test\*.jpg"
#arquivos = glob.glob(targetPattern)

pasta = r"CSGO_images//test//"
fileExt = r".jpg"
fileExt_1 = r".JPG"
arquivos  = [_ for _ in os.listdir(pasta) if _.endswith(fileExt) or _.endswith(fileExt_1)]
arquivos

['1outa1.jpg',
 '1outa100.jpg',
 '1outa112.jpg',
 '1outa153.jpg',
 '1outa167.jpg',
 '1outa174.jpg',
 '1outa180.jpg',
 '1outa185.jpg',
 '1outa20.jpg',
 '1outa211.jpg',
 '1outa227.jpg',
 '1outa238.jpg',
 '1outa239.jpg',
 '1outa265.jpg',
 '1outa27.jpg',
 '1outa270.jpg',
 '1outa288.jpg',
 '1outa293.jpg',
 '1outa300.jpg',
 '1outa32.jpg',
 '1outa323.jpg',
 '1outa358.jpg',
 '1outa376.jpg',
 '1outa381.jpg',
 '1outa410.jpg',
 '1outa421.jpg',
 '1outa44.jpg',
 '1outa444.jpg',
 '1outa447.jpg',
 '1outa448.jpg',
 '1outa452.jpg',
 '1outa454.jpg',
 '1outa460.jpg',
 '1outa462.jpg',
 '1outa513.jpg',
 '1outa524.jpg',
 '1outa539.jpg',
 '1outa557.jpg',
 '1outa56.jpg',
 '1outa573.jpg',
 '1outa580.jpg',
 '1outa584.jpg',
 '1outa592.jpg',
 '1outa605.jpg',
 '1outa608.jpg',
 '1outa61.jpg',
 '1outa615.jpg',
 '1outa620.jpg',
 '1outa680.jpg',
 '1outa710.jpg',
 '1outa711.jpg',
 '1outa716.jpg',
 '1outa724.jpg',
 '1outa735.jpg',
 '1outa740.jpg',
 '1outa749.jpg',
 '1outa75.jpg',
 '1outa751.jpg',
 '1outa772.jpg',
 '1out

In [23]:
test_csv = pd.read_csv('test_labels_1.csv')
arr = np.array(test_csv)
arr

array([['1outa1.jpg', 2, 0.38888888899999996, 0.484375,
        0.7517361109999999, 0.59765625],
       ['1outa100.jpg', 2, 0.42708333299999995, 0.436523438,
        0.7795138890000001, 0.6171875],
       ['1outa112.jpg', 2, 0.43576388899999996, 0.47265625, 0.84375,
        0.5859375],
       ...,
       ['IMG_9365.JPG', 0, nan, nan, nan, nan],
       ['IMG_9366.JPG', 0, nan, nan, nan, nan],
       ['IMG_9367.JPG', 0, nan, nan, nan, nan]], dtype=object)

In [24]:
import cv2
previsoes = []
l = 0

Verdadeiro_positivo = 0
Falso_positivo = 0
Falso_negativo = 0
Verdadeiro_negativo = 0

for w in arquivos:
    image_np = cv2.imread(pasta + w)
    print("step:", l)
    l = l + 1 
    try:
        with detection_graph.as_default():
                with tf.compat.v1.Session() as sess:
                    
                    # Get handles to input and output tensors
                    ops = tf.compat.v1.get_default_graph().get_operations()
                    all_tensor_names = {output.name for op in ops for output in op.outputs}
                    tensor_dict = {}
                    
                    for key in [
                      'num_detections', 'detection_boxes', 'detection_scores',
                      'detection_classes', 'detection_masks'
                    ]:
                        tensor_name = key + ':0'
                        if tensor_name in all_tensor_names:
                            tensor_dict[key] = tf.compat.v1.get_default_graph().get_tensor_by_name(
                          tensor_name)
                            #print(tensor_dict[key], "\n")

                    #ret, image_np = cap.read()
                    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
                    image_np_expanded = np.expand_dims(image_np, axis=0)
                    #print("image_np_expanded", "\n")
                    # Actual detection.
                    output_dict = run_inference_for_single_image(image_np, detection_graph)
                    #print(output_dict, "\n")

                    test_result = tf.greater(output_dict['detection_scores'], [0.50])
                    j = 0

                    linha = []
                    previsoes_2 = []
                    linha.append(w)
                    
                    
                    for i in test_result.eval():
                        if i == True:
                            linha = []
                            linha.append(w)
                            linha.append(output_dict['detection_classes'][j])
                            linha.append(output_dict['detection_boxes'][j][0])
                            linha.append(output_dict['detection_boxes'][j][1])
                            linha.append(output_dict['detection_boxes'][j][2])
                            linha.append(output_dict['detection_boxes'][j][3])
                            #print(linha_1, "\n")
                            
                            #linha.append(linha_1)
                            #linha.append(linha_2)

                            previsoes.append(linha)
                            previsoes_2.append(linha)
                            print(linha, "\n")
                            
                        j = j + 1
                        
                    index = np.argwhere(arr == w)
                    print("index")
                    print("1", index)
                    print(len(index))
                    print("arr1", arr[index[0][0]][1])

                    if len(previsoes_2) != 0:       
                        
                        matrix_resultado = []
                        q = []
                        n = 0
                        for i in index:
                            k = []
                            m = 0
                            for g in previsoes_2:
                                Erro_cord = []
                                #print("if2", arr[i[0]][1] == g[1], "identificação de classe", arr[i[0]][1], g[1])
                                if arr[i[0]][1] == g[1]:
                                    Erro_cord.append(arr[i[0]][2] - g[2])
                                    Erro_cord.append(arr[i[0]][3] - g[3])
                                    Erro_cord.append(arr[i[0]][4] - g[4])
                                    Erro_cord.append(arr[i[0]][5] - g[5])
                                    erro_1 = tf.greater(Erro_cord, [0.05])
                                    erro_2 = tf.less(Erro_cord, [-0.05])
                                    with tf.compat.v1.Session() as sess:
                                        if any(o == True for o in erro_1.eval()) or any(o == True for o in erro_2.eval()):
                                            #print("Coordenada errada")
                                            k.append(False)
                                            
                                        else:                           
                                            #print("acetou")
                                            k.append(True)
                                            
                                else:
                                    #print("Classe errada")
                                    k.append(False)
                                m = m + 1        
                            q.append(k)
                            print("k", k)
                                
                            n = n + 1
                        print("q", q)
                        print("m", m)
                        print("n", n)
                        
                        df = pd.DataFrame(q)
                        for f in range(0, m, 1):
                            print("F1", f)
                            c = np.all(df[f] == False)
                            print("Coluna", f, c)
                            if c:
                                Falso_positivo = Falso_positivo + 1
                        
                        for f in range(0, n, 1):
                            print("F2", f)
                            c = np.any(q[f])
                            print("Linha", f, c)
                            if c:
                                Verdadeiro_positivo = Verdadeiro_positivo + 1
                            else:
                                Falso_negativo = Falso_negativo + 1
                    
                    else:
                        if arr[index[0][0]][1] == 0:
                            Verdadeiro_negativo = Verdadeiro_negativo + 1
                    
                        else:                        
                            Falso_negativo = Falso_negativo + 1
                            #print("não encontrou nada", "Falso_negativo", Falso_negativo)
                    
                    print("Falso_negativo", Falso_negativo)
                    print("Verdadeiro_positivo", Verdadeiro_positivo)
                    print("Falso_positivo", Falso_positivo)
                    print("Verdadeiro_negativo", Verdadeiro_negativo)  
                    
                    # Visualization of the results of a detection.
                    vis_util.visualize_boxes_and_labels_on_image_array(
                        image_np,
                        output_dict['detection_boxes'],
                        output_dict['detection_classes'],
                        output_dict['detection_scores'],
                        category_index,
                        instance_masks=output_dict.get('detection_masks'),
                        use_normalized_coordinates=True,
                        line_thickness=3)
                    #cv2.imshow('object_detection', cv2.resize(image_np, (800, 600)))
                    #while True:
                        #if cv2.waitKey(25) & 0xFF == ord('q'):
                            #cv2.destroyAllWindows()
                            #break

    except Exception as e:
        print(e)
        #cap.release()

print("Falso_negativo", Falso_negativo)
print("Verdadeiro_positivo", Verdadeiro_positivo)
print("Falso_positivo", Falso_positivo)
print("Verdadeiro_negativo", Verdadeiro_negativo)  


step: 0
['1outa1.jpg', 2, 0.38430053, 0.48116565, 0.76235, 0.5974591] 

index
1 [[0 0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 0
Verdadeiro_positivo 1
Falso_positivo 0
Verdadeiro_negativo 0
step: 1
['1outa100.jpg', 2, 0.40817744, 0.4340309, 0.766313, 0.6208662] 

index
1 [[1 0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 0
Verdadeiro_positivo 2
Falso_positivo 0
Verdadeiro_negativo 0
step: 2
['1outa112.jpg', 2, 0.42388153, 0.4688506, 0.8406563, 0.5905974] 

index
1 [[2 0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 0
Verdadeiro_positivo 3
Falso_positivo 0
Verdadeiro_negativo 0
step: 3
['1outa153.jpg', 2, 0.46331567, 0.38293976, 0.7089345, 0.5939287] 

index
1 [[3 0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 0
Verdadeiro_positivo 4
Falso_positivo 0
Verdadeiro_negativo 0
step: 4
['1outa167.jpg', 2,

['1outa462.jpg', 2, 0.37351626, 0.4156854, 0.6844906, 0.6050515] 

index
1 [[33  0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 0
Verdadeiro_positivo 34
Falso_positivo 1
Verdadeiro_negativo 0
step: 34
['1outa513.jpg', 2, 0.516129, 0.44096765, 0.7808424, 0.6482221] 

index
1 [[34  0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 0
Verdadeiro_positivo 35
Falso_positivo 1
Verdadeiro_negativo 0
step: 35
['1outa524.jpg', 2, 0.48329753, 0.4263042, 0.78920573, 0.6405906] 

index
1 [[35  0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 0
Verdadeiro_positivo 36
Falso_positivo 1
Verdadeiro_negativo 0
step: 36
['1outa539.jpg', 2, 0.49361116, 0.4256522, 0.80154675, 0.6147534] 

index
1 [[36  0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 0
Verdadeiro_positivo 37
Falso_positivo 1
Verdadeiro_negativo 0
step: 37
['1outa

['1outa840.jpg', 2, 0.34211954, 0.44714534, 0.7414433, 0.56502724] 

index
1 [[68  0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 4
Verdadeiro_positivo 65
Falso_positivo 2
Verdadeiro_negativo 0
step: 69
index
1 [[69  0]]
1
arr1 2
Falso_negativo 5
Verdadeiro_positivo 65
Falso_positivo 2
Verdadeiro_negativo 0
step: 70
['1outa854.jpg', 2, 0.37057784, 0.4025012, 0.681569, 0.56515205] 

index
1 [[70  0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 5
Verdadeiro_positivo 66
Falso_positivo 2
Verdadeiro_negativo 0
step: 71
['1outa856.jpg', 2, 0.36527497, 0.39815003, 0.7075469, 0.55516833] 

index
1 [[71  0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 5
Verdadeiro_positivo 67
Falso_positivo 2
Verdadeiro_negativo 0
step: 72
['1outa875.jpg', 2, 0.34654808, 0.41286403, 0.6736367, 0.57703495] 

index
1 [[72  0]]
1
arr1 2
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0

['1outb314.jpg', 1, 0.38176373, 0.4695574, 0.745049, 0.5890429] 

index
1 [[102   0]]
1
arr1 1
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 6
Verdadeiro_positivo 97
Falso_positivo 2
Verdadeiro_negativo 0
step: 103
['1outb344.jpg', 1, 0.38901812, 0.51286846, 0.7903591, 0.59811527] 

index
1 [[103   0]]
1
arr1 1
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 6
Verdadeiro_positivo 98
Falso_positivo 2
Verdadeiro_negativo 0
step: 104
['1outb35.jpg', 1, 0.39916974, 0.4272529, 0.7382166, 0.6238146] 

index
1 [[104   0]]
1
arr1 1
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 6
Verdadeiro_positivo 99
Falso_positivo 2
Verdadeiro_negativo 0
step: 105
['1outb373.jpg', 1, 0.4161075, 0.44324017, 0.76995814, 0.6155381] 

index
1 [[105   0]]
1
arr1 1
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 6
Verdadeiro_positivo 100
Falso_positivo 2
Verdadeiro_negativo 0
step

['1outb684.jpg', 1, 0.34453183, 0.46014744, 0.69859344, 0.6305371] 

index
1 [[135   0]]
1
arr1 1
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 6
Verdadeiro_positivo 130
Falso_positivo 2
Verdadeiro_negativo 0
step: 136
['1outb686.jpg', 1, 0.3595639, 0.4561373, 0.69252104, 0.627507] 

index
1 [[136   0]]
1
arr1 1
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 6
Verdadeiro_positivo 131
Falso_positivo 2
Verdadeiro_negativo 0
step: 137
['1outb713.jpg', 1, 0.34707657, 0.49601075, 0.72593486, 0.6674193] 

index
1 [[137   0]]
1
arr1 1
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 6
Verdadeiro_positivo 132
Falso_positivo 2
Verdadeiro_negativo 0
step: 138
['1outb714.jpg', 1, 0.36915803, 0.50321364, 0.7211206, 0.67344975] 

index
1 [[138   0]]
1
arr1 1
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 6
Verdadeiro_positivo 133
Falso_positivo 2
Verdadeiro_negativo

...

['1outd157.jpg', 4, 0.34268582, 0.5402159, 0.69882095, 0.7788277] 

index
1 [[235   0]]
1
arr1 4
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 9
Verdadeiro_positivo 227
Falso_positivo 3
Verdadeiro_negativo 0
step: 236
['1outd164.jpg', 4, 0.23722672, 0.53542435, 0.6876467, 0.78477824] 

index
1 [[236   0]]
1
arr1 4
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 9
Verdadeiro_positivo 228
Falso_positivo 3
Verdadeiro_negativo 0
step: 237
['1outd168.jpg', 4, 0.3377738, 0.5354594, 0.64076567, 0.7988008] 

index
1 [[237   0]]
1
arr1 4
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 9
Verdadeiro_positivo 229
Falso_positivo 3
Verdadeiro_negativo 0
step: 238
['1outd184.jpg', 4, 0.161185, 0.51843977, 0.641338, 0.75705874] 

['1outd184.jpg', 3, 0.5931554, 0.6722603, 0.993085, 1.0] 

index
1 [[238   0]]
1
arr1 4
k [True, False]
q [[True, False]]
m 2
n 1
F1 0
Coluna 0 False
F1 1
Coluna 1 True
F2 0
L

['1outd93.jpg', 4, 0.5315435, 0.56157744, 0.8246361, 0.8491005] 

index
1 [[264   0]]
1
arr1 4
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 9
Verdadeiro_positivo 256
Falso_positivo 13
Verdadeiro_negativo 0
step: 265
['1outd97.jpg', 4, 0.51497513, 0.55537933, 0.82514066, 0.8395173] 

index
1 [[265   0]]
1
arr1 4
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 9
Verdadeiro_positivo 257
Falso_positivo 13
Verdadeiro_negativo 0
step: 266
['1oute1.jpg', 3, 0.08327466, 0.51275533, 0.99813944, 0.81987554] 

index
1 [[266   0]]
1
arr1 3
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 9
Verdadeiro_positivo 258
Falso_positivo 13
Verdadeiro_negativo 0
step: 267
index
1 [[267   0]]
1
arr1 3
Falso_negativo 10
Verdadeiro_positivo 258
Falso_positivo 13
Verdadeiro_negativo 0
step: 268
['1oute105.jpg', 3, 0.038770318, 0.50871074, 0.90233797, 0.86491585] 

index
1 [[268   0]]
1
arr1 3
k [True]
q [[True]]

['1oute373.jpg', 4, 0.13480562, 0.32629037, 1.0, 0.7602762] 

['1oute373.jpg', 3, 0.13424933, 0.31367454, 0.9848894, 0.7496911] 

index
1 [[302   0]]
1
arr1 3
k [False, True]
q [[False, True]]
m 2
n 1
F1 0
Coluna 0 True
F1 1
Coluna 1 False
F2 0
Linha 0 True
Falso_negativo 31
Verdadeiro_positivo 272
Falso_positivo 26
Verdadeiro_negativo 0
step: 303
['1oute389.jpg', 4, 0.009410739, 0.37535924, 0.64810336, 0.9389623] 

index
1 [[303   0]]
1
arr1 3
k [False]
q [[False]]
m 1
n 1
F1 0
Coluna 0 True
F2 0
Linha 0 False
Falso_negativo 32
Verdadeiro_positivo 272
Falso_positivo 27
Verdadeiro_negativo 0
step: 304
['1oute390.jpg', 3, 0.023730338, 0.3195889, 0.6954655, 0.7593672] 

index
1 [[304   0]]
1
arr1 3
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 32
Verdadeiro_positivo 273
Falso_positivo 27
Verdadeiro_negativo 0
step: 305
['1oute394.jpg', 3, 0.06427285, 0.31292424, 0.66899633, 0.753353] 

['1oute394.jpg', 4, 0.109550446, 0.3051384, 0.64948, 0.75544316] 

i

k [False, True]
q [[False, True]]
m 2
n 1
F1 0
Coluna 0 True
F1 1
Coluna 1 False
F2 0
Linha 0 True
Falso_negativo 41
Verdadeiro_positivo 297
Falso_positivo 32
Verdadeiro_negativo 0
step: 337
['IMG_9009.JPG', 5, 0.24437107, 0.2872295, 0.6918473, 0.8842832] 

index
1 [[338   0]]
1
arr1 5
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 41
Verdadeiro_positivo 298
Falso_positivo 32
Verdadeiro_negativo 0
step: 338
['IMG_9028.JPG', 4, 0.28988424, 0.25574666, 0.81878996, 0.55297863] 

index
1 [[339   0]]
1
arr1 4
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 41
Verdadeiro_positivo 299
Falso_positivo 32
Verdadeiro_negativo 0
step: 339
['IMG_9036.JPG', 4, 0.3339548, 0.43181553, 0.7730892, 0.7838762] 

index
1 [[340   0]]
1
arr1 4
k [True]
q [[True]]
m 1
n 1
F1 0
Coluna 0 False
F2 0
Linha 0 True
Falso_negativo 41
Verdadeiro_positivo 300
Falso_positivo 32
Verdadeiro_negativo 0
step: 340
['IMG_9137.JPG', 5, 0.34902596, 0.25749236, 

['IMG_9366.JPG', 2, 0.25860435, 0.302047, 0.69770133, 0.58757764] 

index
1 [[376   0]]
1
arr1 0
k [False]
q [[False]]
m 1
n 1
F1 0
Coluna 0 True
F2 0
Linha 0 False
Falso_negativo 61
Verdadeiro_positivo 301
Falso_positivo 55
Verdadeiro_negativo 15
step: 376
index
1 [[377   0]]
1
arr1 0
Falso_negativo 61
Verdadeiro_positivo 301
Falso_positivo 55
Verdadeiro_negativo 16
Falso_negativo 61
Verdadeiro_positivo 301
Falso_positivo 55
Verdadeiro_negativo 16


In [25]:
presisao = Verdadeiro_positivo/(Verdadeiro_positivo+Falso_positivo)
presisao

0.8455056179775281

In [26]:
seletividade = Verdadeiro_positivo/(Verdadeiro_positivo+Falso_negativo)
seletividade

0.8314917127071824

In [27]:
especificidade = Verdadeiro_negativo/(Verdadeiro_negativo+Falso_positivo)
especificidade

0.22535211267605634

In [28]:
acuracidade = (Verdadeiro_positivo+Verdadeiro_negativo)/(Verdadeiro_positivo+Verdadeiro_negativo+Falso_negativo+Falso_positivo)
acuracidade

0.7321016166281755